In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117
!pip install git+https://github.com/huggingface/transformers
!pip install transformers
!pip install datasets
!pip install sklearn
!pip install matplotlib
!pip install huggingface-hub


In [1]:
import huggingface_hub
huggingface_hub.notebook_login()

In [ ]:
import torch
import gc
import math
import pandas as pd
from datasets import Dataset
from torch.utils.data import random_split, DataLoader
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
model_checkpoint = "distilgpt2"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).cuda()
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
descriptions = pd.read_csv("descriptions_2.csv")
description_list = list(descriptions["Description"])

kai kurie lb ilgi ir reik bilekiek padint, tai atmetu ilgus

In [ ]:
shorter_descriptions = []
for description in description_list:
    if len(tokenizer.encode(description)) < 100:
        shorter_descriptions.append(description)

In [ ]:
class DescriptionDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer(txt, truncation=True, max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
dataset = DescriptionDataset(shorter_descriptions, tokenizer, max_length=100)

In [ ]:
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

In [ ]:
model_name = "distilgpt2"

training_args = TrainingArguments(
    f"{model_name}_finetuned_hacks",
    num_train_epochs=10,
    evaluation_strategy = "epoch",
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset,
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])})

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.push_to_hub()